In [1]:
import tensorflow as tf
from tensorflow.python.ops import rnn, rnn_cell
import numpy as np
import os
import librosa
import matplotlib.pyplot as plt
import csv

# Hyperparameter

In [7]:
input_size = 499
hidden_layer = 5

sampling_rate = 200
striding = 100
train_steps = 1000

# Onset-time data

In [ ]:
def seizure_time_parser(onset_times,patient_number,data_set_number):
    temp = str(onset_times[patient_number-1][data_set_number])
    curr_onset_time = []
    while temp.find('/')!=-1:
        curr_onset_time.append(int(temp[0:temp.find('/')]))
        temp=temp[temp.find('/')+1:]
    curr_onset_time.append(int(float(temp)))
    curr_onset_time=np.array(curr_onset_time)
    return curr_onset_time

In [ ]:
seizure_file = open('/home/maestroj/medical_analysis/project2/seizure_times.csv','r',newline='')
reader = csv.reader(seizure_file,delimiter=',')
onset_times_temp=[]

for row in reader:
    onset_times_temp.append(row)
    
onset_times = []
total_patient = len(onset_times_temp)
event_number = np.zeros(total_patient,dtype=np.int32)
#number of events of (i+1) patient = event_number[i]

for i in range(total_patient):
    event_number[i] = len(onset_times_temp[i])-1

for p in range(total_patient):
    temp = []
    for d in range(event_number[p]):
        temp.append(seizure_time_parser(onset_times_temp,p+1,d+1))
    onset_times.append(temp)

In [ ]:
print("The number of datasets of patients")
print(event_number)
print("1st patient's seizure time for each data_set")
print(onset_times[0][1])
print("9th patient's seizure time for each data_set")
print(onset_times[8])

In [ ]:
def stft_load(patient_number, dataset_number):
    input_folder_path = '/home/maestroj/medical_analysis/stft_data/'
    input_data_name = '{}_{}.csv'.format(patient_number,dataset_number)
    input_data_path = os.path.join(input_folder_path,input_data_name)
    input_file = open(input_data_path,'r',newline='')
    reader = csv.reader(input_file, delimiter=',')
    print("{} is loaded".format(input_data_name))
    temp = []
    for row in reader:
        temp.append(row)
    try:
        temp = np.array(temp, dtype=np.float32)
    except: 
        for i in range(len(temp)):
            for j in range(len(temp[0])):
                temp[i][j]=abs(complex(temp[i][j].replace('i','j')))
        temp = np.array(temp, dtype=np.float32)
        temp = np.transpose(temp)
    return temp

In [ ]:
def stft_normal(dataset_number):
    input_folder_path = '/home/maestroj/medical_analysis/stft_normal/'
    input_data_name = '{}.csv'.format(dataset_number)
    input_data_path = os.path.join(input_folder_path,input_data_name)
    input_file = open(input_data_path,'r',newline='')
    reader = csv.reader(input_file, delimiter=',')
    print("{} is loaded".format(input_data_name))
    temp = []
    for row in reader:
        temp.append(row)
    try:
        temp = np.array(temp, dtype=np.float32)
    except: 
        for i in range(len(temp)):
            for j in range(len(temp[0])):
                temp[i][j]=abs(complex(temp[i][j].replace('i','j')))
        temp = np.array(temp, dtype=np.float32)
        temp = np.transpose(temp)
    return temp

In [ ]:
def time_axis_maker(array, sec_per_cell,init = 0):
    final = init + sec_per_cell*(len(array)-1)
    return np.linspace(init,final,len(array))

In [8]:
restorer = tf.train.import_meta_graph('auto.meta')
sess = tf.Session()
restorer.restore(sess, tf.train.latest_checkpoint('./'))
inputs = tf.get_collection("input")
X = inputs[0]
outputs = tf.get_collection("hypo")
H = outputs[0]

# Train_set and test_set generator

In [ ]:
train_input_data = []
train_output_data = []
test_input_data = []
test_output_data = []
one_hot = np.eye(4)
for i in range(4):
    train_index = 0.7*nsounds[i]
    for j in range(1,nsounds[i]+1):
        try :
            result = music_loader(i,j)
            y = result['y']
            D = librosa.stft(y,win_length=win_length, hop_length= hop_length)
            D = np.transpose(abs(D))
            if D.shape[0] < train_length:
                continue
                
            if j < train_index:
                train_input_data.append(sess.run(H, feed_dict={X : D[0:train_length]}))
                train_output_data.append(one_hot[i])
            else :
                test_input_data.append(sess.run(H, feed_dict={X : D[0:train_length]}))
                test_output_data.append(one_hot[i])
        except FileNotFoundError:
            print("File not found")
test_input_data = np.array(test_input_data)
test_output_data = np.array(test_output_data)
train_input_data = np.array(train_input_data)
train_output_data = np.array(train_output_data)

In [13]:
print("Size of train and test set")
print(train_input_data.shape)
print(train_output_data.shape)
print(test_input_data.shape)
print(test_output_data.shape)

(144, 40, 10)
(144, 4)
(62, 40, 10)
(62, 4)
